<a href="https://colab.research.google.com/github/YasiruMethmal123/See-People/blob/master/SeePeopleAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas openpyxl sentence-transformers scikit-learn numpy -q

import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import pickle
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv("demograpic_district wise.csv", encoding="latin1")
df.head()

,OBJECTID,PROVINCE_N,DISTRICT_N,DSD_N,DSD_C,GND_N,GND_NO,GND_C,PROVINCE_C,DISTRICT_C,...,PPROJ_20,PPROJ_21,PPROJ_22,NOTES_,SOURCE_,CreateDATE,CreateBY,EditDATE,Shape_Leng,Shape_Area
0,1,Central,Kandy,Akurana,2109,Akurana,564,120,2,1,...,2734.0,2759.0,2784.0,2012 Census data. TOT_POP are the population f...,DCS LKA,2016-06-28,WFP/OCHA,NaN,0.034231,0.000042
1,2,Central,Kandy,Akurana,2109,Alawathugoda,583,25,2,1,...,4100.0,4137.0,4175.0,2012 Census data. TOT_POP are the population f...,DCS LKA,2016-06-28,WFP/OCHA,NaN,0.055164,0.000127
2,3,Central,Kandy,Akurana,2109,Arambepola,584,40,2,1,...,1391.0,1404.0,1417.0,2012 Census data. TOT_POP are the population f...,DCS LKA,2016-06-28,WFP/OCHA,NaN,0.043929,0.000085
3,4,Central,Kandy,Akurana,2109,Balakaduwa,585,30,2,1,...,1081.0,1090.0,1100.0,2012 Census data. TOT_POP are the population f...,DCS LKA,2016-06-28,WFP/OCHA,NaN,0.040703,0.000032
4,5,Central,Kandy,Akurana,2109,Bulugohothenna,559,95,2,1,...,2328.0,2349.0,2370.0,2012 Census data. TOT_POP are the population f...,DCS LKA,2016-06-28,WFP/OCHA,NaN,0.033539,0.000048


In [7]:
df.isna().sum().sum()

np.int64(14132)

EDA Summary

In [8]:

print(f"Provinces: {df['PROVINCE_N'].nunique()} | Districts: {df['DISTRICT_N'].nunique()} | GN Divisions: {len(df)}")
print(f"Population range: {df['TOT_POP'].min():,} → {df['TOT_POP'].max():,}")
print(f"Missing TOT_POP: {df['TOT_POP'].isna().sum()} | Missing MALE/FEMALE: {df['MALE'].isna().sum()}")

Provinces: 9 | Districts: 25 | GN Divisions: 14022
Population range: 0 → 27,776
Missing TOT_POP: 0 | Missing MALE/FEMALE: 0


Preprocessing and Cleaning

In [9]:
df = df.copy()

In [10]:
cols_numeric = ['TOT_POP', 'MALE', 'FEMALE', 'area']
for col in cols_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [11]:
before = len(df)
df = df.dropna(subset=['TOT_POP', 'MALE', 'FEMALE', 'GND_N', 'DISTRICT_N', 'PROVINCE_N'])
after = len(df)
print(f"Cleaned data: {before} → {after} rows ({before-after} removed)")

Cleaned data: 14022 → 14022 rows (0 removed)


Feature Engineering

In [12]:
df['male_pct']       = (df['MALE'] / df['TOT_POP'] * 100).round(2)
df['female_pct']     = (df['FEMALE'] / df['TOT_POP'] * 100).round(2)
df['sex_ratio']      = (df['MALE'] / df['FEMALE'] * 100).round(1)  # males per 100 females
df['pop_density']    = (df['TOT_POP'] / df['area']).round(0).astype(int)
df['log_pop']        = np.log10(df['TOT_POP'] + 1)

In [13]:
df['size_category'] = pd.cut(df['TOT_POP'],
                             bins=[0, 1000, 5000, 10000, 50000, float('inf')],
                             labels=['Tiny (<1k)', 'Small (1-5k)', 'Medium (5-10k)', 'Large (10-50k)', 'Very Large (>50k)'])

# Density category
df['density_level'] = pd.cut(df['pop_density'],
                             bins=[0, 100, 500, 1000, 5000, float('inf')],
                             labels=['Rural', 'Semi-Urban', 'Urban', 'Dense', 'Very Dense'])

CREATE RICH TEXT DESCRIPTIONS (for NLP)

In [14]:
def make_description(row):
    return (f"{row['GND_N']} is a {row['size_category']} GN division in {row['DSD_N']}, "
            f"{row['DISTRICT_N']} district, {row['PROVINCE_N']} province, Sri Lanka. "
            f"It has a population of {int(row['TOT_POP']):,} people "
            f"({int(row['MALE']):,} males, {int(row['FEMALE']):,} females). "
            f"Male percentage: {row['male_pct']}%, female percentage: {row['female_pct']}%. "
            f"Sex ratio: {row['sex_ratio']} males per 100 females. "
            f"Population density: {row['pop_density']:,} people per km² ({row['density_level']}).")

df['description'] = df.apply(make_description, axis=1)

TRAIN NLP SEARCH ENGINE

In [15]:
nlp_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = nlp_model.encode(df['description'].tolist(), batch_size=128, show_progress_bar=True)

print(f"NLP embeddings created: {embeddings.shape}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/110 [00:00<?, ?it/s]

NLP embeddings created: (14022, 384)


Model Export

In [16]:
final_model = {
    'data': df.reset_index(drop=True),
    'embeddings': embeddings,
    'nlp_model': nlp_model,
    'columns_for_display': ['GND_N', 'DSD_N', 'DISTRICT_N', 'PROVINCE_N',
                           'TOT_POP', 'MALE', 'FEMALE', 'male_pct', 'pop_density', 'sex_ratio', 'density_level']
}

with open('model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

print("\nMODEL SUCCESSFULLY CREATED!")
print("File: model.pkl")
print("Size:", round(len(pickle.dumps(final_model)) / 1024 / 1024, 2), "MB")


MODEL SUCCESSFULLY CREATED!
File: model.pkl
Size: 118.54 MB


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import re

# Load what we just saved
data = final_model['data']
embeddings = final_model['embeddings']
nlp_model = final_model['nlp_model']

def ask(query, top_n=10):
    q_emb = nlp_model.encode([query])
    sims = cosine_similarity(q_emb, embeddings)[0]
    top_idx = np.argsort(sims)[::-1][:100]
    result = data.iloc[top_idx].copy()
    result['score'] = sims[top_idx]

    q = query.lower()

    # Smart ranking rules (same as Streamlit)
    if any(w in q for w in ["top", "highest", "most populated", "largest", "biggest"]):
        result = result.sort_values("TOT_POP", ascending=False)
    if "female" in q or "women" in q:
        result = result.sort_values("female_pct", ascending=False)
    if "male" in q or "men" in q:
        result = result.sort_values("male_pct", ascending=False)
    if "dense" in q:
        result = result.sort_values("pop_density", ascending=False)

    # Population filters
    m = re.search(r"(above|over|>)\s*(\d+)", q)
    if m:
        result = result[result['TOT_POP'] > int(m.group(2))]
        result = result.sort_values("TOT_POP", ascending=False)
    m = re.search(r"(below|under|<)\s*(\d+)", q)
    if m:
        result = result[result['TOT_POP'] < int(m.group(2))]

    result = result.head(top_n)
    display = result[['GND_N', 'DISTRICT_N', 'PROVINCE_N', 'TOT_POP', 'MALE', 'FEMALE',
                      'male_pct', 'pop_density', 'density_level']].copy()
    display['TOT_POP'] = display['TOT_POP'].astype(int).apply(lambda x: f"{x:,}")
    display['MALE'] = display['MALE'].astype(int).apply(lambda x: f"{x:,}")
    display['FEMALE'] = display['FEMALE'].astype(int).apply(lambda x: f"{x:,}")
    return display

# ———— TRY THESE QUERIES ————
queries = [
    "top 20 most populated areas",
    "areas with population above 10000",
    "most female dominated places",
    "densely populated areas in Colombo",
    "small villages under 1000 people in Kandy",
    "places with more males than females"
]

for q in queries:
    print(f"\n🔍 Query: {q}")
    print(ask(q, top_n=8).to_string(index=False))
    print("-" * 80)


🔍 Query: top 20 most populated areas
         GND_N   DISTRICT_N    PROVINCE_N TOT_POP  MALE FEMALE  male_pct  pop_density density_level
 Puttalam East     Puttalam North Western  13,894 6,710  7,184     48.29         4257         Dense
Mirihana North      Colombo       Western   6,193 2,908  3,285     46.96        10405    Very Dense
Mirihana South      Colombo       Western   6,004 2,964  3,040     49.37        10603    Very Dense
          Park Nuwara Eliya       Central   4,510 2,157  2,353     47.83          747         Urban
   Hirana West     Kalutara       Western   4,050 1,943  2,107     47.98         2232         Dense
   Mabima East      Gampaha       Western   3,292 1,664  1,628     50.55         1959         Dense
 Great Western Nuwara Eliya       Central   3,139 1,522  1,617     48.49          362    Semi-Urban
 Madurankuliya     Puttalam North Western   2,871 1,434  1,437     49.95          137    Semi-Urban
--------------------------------------------------------------